In [1]:
device = 'cuda'

In [2]:
import random
import numpy as np
import torch

In [3]:
def set_seed(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed()


In [4]:
# hyperparameters
from sched import scheduler


batch_size = 64
n_worker = 24

image_size = 512
model = 'tf_efficientnetv2_l_in21k'
epochs = 20
lr = 1e-4

loss_fn = "FocalLoss"
focal_alpha = 0.6
focal_gamma = 1.8
aux_loss = 'binary_cross_entropy'

loss1_coef = 1
optmizer = 'AdamW'
scheduler = 'CosineAnnealingWarmRestarts'




In [75]:
import pandas as pd
train_label_image = pd.read_csv('./siim-covid19-detection/train_image_level.csv')
train_label_study = pd.read_csv('./siim-covid19-detection/train_study_level.csv')

# train_label

In [104]:
study_id = set()
boxes_study_id= set()
uniqued_train_label_image = pd.DataFrame(columns = train_label_image.columns)

for idx,image_id,boxes,label,StudyInstanceUID in train_label_image.itertuples():
    # break
    if StudyInstanceUID not in study_id:
        study_id.add(StudyInstanceUID)
        # check if opacity in label
        if label.find('none') == -1:
            boxes_study_id.add(StudyInstanceUID)
        # print(uniqued_train_label_image.columns)
        # print(train_label_image.iloc[idx])
        uniqued_train_label_image = uniqued_train_label_image.append(train_label_image.iloc[idx])
        # print(uniqued_train_label_image.columns)
        # break
    else:
        if StudyInstanceUID in boxes_study_id:
            continue
        elif label.find('none') == -1:
            boxes_study_id.add(StudyInstanceUID)
            uniqued_train_label_image.loc[uniqued_train_label_image['StudyInstanceUID'] == StudyInstanceUID] = [image_id,boxes,label,StudyInstanceUID]

uniqued_train_label_image = uniqued_train_label_image.reset_index(drop=True)

    
        

In [109]:
train_label_study['StudyInstanceUID'] = train_label_study['id'].apply(lambda x: x.replace('_study',''))

In [111]:
train_df = pd.merge(uniqued_train_label_image,train_label_study,on='StudyInstanceUID')

In [ ]:
train_df.rename(columns={'id_x':'id_image'},inplace=True)
train_df.drop(['id_y'],axis=1,inplace=True)


In [122]:
train_df['id_image'] = train_df['id_image'].apply(lambda x: x.replace('_image',''))

In [123]:
train_df

,id_image,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0
...,...,...,...,...,...,...,...,...
6049,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,1,0,0,0
6050,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,1,0,0,0
6051,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,0,1,0,0
6052,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,0,1,0,0


In [125]:
train_df['filepath'] = './data/train/images/'+train_df['id_image']+'.png'

In [127]:
from sklearn.model_selection import GroupKFold
folds = GroupKFold(n_splits=5)

In [130]:
# add column for fold
train_df['fold_id'] = -1
for fold_id,(train_idx,valid_idx) in enumerate(folds.split(train_df,groups=train_df.StudyInstanceUID.tolist())):
    train_df.loc[valid_idx,'fold_id'] = fold_id

In [131]:
train_df

,id_image,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,filepath,fold_id
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,./data/train/images/000a312787f2.png,3
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0,./data/train/images/000c3a3f293f.png,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0,./data/train/images/0012ff7358bc.png,2
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1,./data/train/images/001398f4ff4f.png,2
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0,./data/train/images/001bd15d1891.png,4
...,...,...,...,...,...,...,...,...,...,...
6049,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,1,0,0,0,./data/train/images/ffcc6edd9445.png,4
6050,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,1,0,0,0,./data/train/images/ffd91a2c4ca0.png,1
6051,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,0,1,0,0,./data/train/images/ffd9b6cf2961.png,0
6052,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,0,1,0,0,./data/train/images/ffdc682f7680.png,3


In [53]:
train_label_image.columns

Index(['id', 'boxes', 'label', 'StudyInstanceUID'], dtype='object')

In [51]:
uniqued_train_label_image = pd.DataFrame(columns = train_label_image.columns)